# Lesson 3: Agent Management & Lifecycle

🟢 Beginner · ⏱ 20 min

---

Agents are persistent resources in Lyzr — they live in the cloud and can be retrieved, modified, and deleted. This lesson covers the full CRUD lifecycle.

**What you'll learn:**
- List and retrieve agents from your account
- Update an agent's configuration without recreating it
- Clone an agent to create independent variants
- Safely delete agents when you're done

## Prerequisites

Before running this notebook, make sure you have:

- Completed **Lesson 1** — Studio init, `create_agent`, and `run`
- Completed **Lesson 2** — Providers and models
- Your `LYZR_API_KEY` set as an environment variable (or ready to paste in the setup cell)

In [ ]:
!pip install lyzr-adk -q

In [ ]:
import os
from lyzr import Studio

API_KEY = os.getenv("LYZR_API_KEY", "YOUR_LYZR_API_KEY")
studio = Studio(api_key=API_KEY)
print("Studio ready!")

## 1. Listing All Your Agents

`studio.list_agents()` returns all agents currently in your account. Each agent object exposes at minimum:

| Attribute | Description |
|-----------|-------------|
| `agent.id` | Unique persistent identifier (string) |
| `agent.name` | Human-readable name you assigned |

This is useful for auditing what's in your account and for recovering agent IDs you may have forgotten.

In [ ]:
agents = studio.list_agents()
print(f"You have {len(agents)} agent(s) in your account:")
for a in agents:
    print(f"  • {a.name} — ID: {a.id}")

## 2. Creating an Agent and Saving Its ID

When you create an agent, Lyzr assigns it a unique `id`. This ID is the persistent handle for the agent — it stays the same across updates and sessions.

**Best practice:** Save `agent.id` to a variable (or to disk/config) if you plan to retrieve the agent in a later session.

In [ ]:
agent = studio.create_agent(
    name="Lifecycle Demo",
    provider="openai/gpt-4o",
    role="Demo assistant",
    goal="Demonstrate agent lifecycle operations",
    instructions="Be brief. Answer in one sentence."
)

agent_id = agent.id  # Save this — we'll use it to retrieve the agent
print(f"Created agent: {agent.name}")
print(f"Agent ID: {agent_id}")

## 3. Retrieving an Agent by ID

`studio.get_agent(agent_id)` fetches a specific agent from the cloud using its ID. This is the pattern you'll use when:

- Resuming work in a new notebook session
- Sharing an agent ID with a teammate
- Building applications that reference a pre-created agent

The returned object is fully functional — you can call `.run()`, `.update()`, `.clone()`, and `.delete()` on it.

In [ ]:
# Simulate retrieving the agent later using just the ID
retrieved_agent = studio.get_agent(agent_id)
print(f"Retrieved: {retrieved_agent.name}")
print(f"Same agent? {retrieved_agent.id == agent_id}")

## 4. Updating an Agent

`agent.update(...)` modifies the agent's configuration **in place** — no new agent is created, and the ID stays the same. You can update any combination of:

- `name` — rename the agent
- `instructions` — change behaviour
- `provider` — switch the underlying model
- `role`, `goal` — adjust the agent's purpose

Updates take effect immediately on the next `.run()` call.

In [ ]:
# Update the agent's instructions to be more detailed
agent.update(
    name="Lifecycle Demo v2",
    instructions="Be very brief. Maximum one sentence. Start with 'In short:'"
)

response = agent.run("Explain gravity.")
print(response.response)

## 5. Cloning an Agent

`agent.clone()` creates an **independent copy** of the agent with a new ID. The original and the clone are completely separate after cloning — changes to one do not affect the other.

**Common use cases:**
- A/B testing different instruction styles
- Creating a "production" copy while keeping a "dev" copy for experimentation
- Branching off a base agent for multiple specialised variants

In [ ]:
clone = agent.clone()
print(f"Original ID: {agent.id}")
print(f"Clone ID:    {clone.id}")
print(f"Clone name:  {clone.name}")

# Modify the clone independently
clone.update(instructions="Be very detailed. Give at least 3 sentences.")
response_original = agent.run("What is gravity?")
response_clone = clone.run("What is gravity?")

print(f"\nOriginal: {response_original.response}")
print(f"\nClone:    {response_clone.response}")

## 6. Deleting an Agent

`agent.delete()` **permanently removes** the agent from your account. There is no undo.

After deletion:
- The agent ID is no longer valid
- Any attempt to call `.run()`, `.update()`, or `.clone()` on the deleted object will raise an error
- `studio.list_agents()` will no longer include it

Always verify deletion with `list_agents()` if you need confirmation.

In [ ]:
clone_id = clone.id
clone.delete()
print(f"Clone {clone_id} deleted.")

# Verify it's gone
remaining_agents = studio.list_agents()
remaining_ids = [a.id for a in remaining_agents]
print(f"Clone still in account: {clone_id in remaining_ids}")  # False

## Common Mistake: Using a Deleted Agent

Once an agent is deleted, the Python object still exists in memory — but calling any method on it will fail because the agent no longer exists on the server.

```python
# After clone.delete() ...
clone.run("...")    # raises an exception
clone.update(...)   # raises an exception
clone.clone()       # raises an exception
```

**Rule of thumb:** Only call `.delete()` when you are completely done with an agent. If you want to keep a backup, clone it first.

In [ ]:
# Using a deleted agent raises an error
try:
    clone.run("Are you still there?")
except Exception as e:
    print(f"Error (expected): {e}")

print("\nAlways delete() only when you're done with the agent permanently.")

## Exercise

Practice the full lifecycle end-to-end:

1. Create an agent for any topic you like
2. Update its instructions
3. Clone it and give the clone different instructions
4. Ask both the same question and compare the responses
5. Delete the clone

Fill in the `...` placeholders below.

In [ ]:
# TODO: Create an agent for any topic you like
my_agent = studio.create_agent(
    name=...,
    provider="openai/gpt-4o",
    role=...,
    goal=...,
    instructions=...
)

# TODO: Update the instructions
my_agent.update(instructions=...)

# TODO: Clone it and give the clone different instructions
my_clone = my_agent.clone()
my_clone.update(instructions=...)

# TODO: Run both with the same question and compare
question = "..."
print("Original:", my_agent.run(question).response)
print("Clone:", my_clone.run(question).response)

# TODO: Delete the clone
my_clone.delete()
print("Clone deleted!")

## Summary

| Operation | Method | Notes |
|-----------|--------|-------|
| List all agents | `studio.list_agents()` | Returns a list of agent objects |
| Retrieve by ID | `studio.get_agent(agent_id)` | Works across sessions; ID never changes |
| Create | `studio.create_agent(...)` | Covered in Lesson 1 |
| Update | `agent.update(...)` | Modifies in place; ID stays the same |
| Clone | `agent.clone()` | Creates an independent copy with a new ID |
| Delete | `agent.delete()` | Permanent — verify with `list_agents()` |

**Key takeaways:**

- `agent.id` is the immutable identifier — save it if you need the agent later
- `update()` is non-destructive; `delete()` is permanent
- Cloning is the safe way to experiment — keep the original, modify the clone
- A deleted Python object still exists in memory but all server calls will fail

## Next Steps

**Lesson 4: Structured Outputs**

So far agents return plain text. In the next lesson you'll learn how to make agents return structured data (JSON schemas, Pydantic models) — essential for building reliable pipelines and integrations.

Continue to `04_structured_outputs.ipynb`.